## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,86.32,67,1,4.00,clear sky
1,Lasa,IT,46.6166,10.7002,59.88,92,43,7.31,scattered clouds
2,Punta Arenas,CL,-53.1500,-70.9167,42.91,93,90,8.05,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,74.68,79,54,14.63,light rain
4,Marsa Matruh,EG,31.3525,27.2453,75.40,78,0,3.44,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 73
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,86.32,67,1,4.00,clear sky
3,Rikitea,PF,-23.1203,-134.9692,74.68,79,54,14.63,light rain
4,Marsa Matruh,EG,31.3525,27.2453,75.40,78,0,3.44,clear sky
5,Hue,VN,16.4667,107.6000,86.11,66,20,5.75,few clouds
7,Vaini,TO,-21.2000,-175.2000,75.36,73,40,11.50,scattered clouds
12,Takoradi,GH,4.8845,-1.7554,75.31,85,90,10.04,overcast clouds
13,Labuhan,ID,-6.8844,112.2051,80.35,70,77,8.66,broken clouds
14,Atuona,PF,-9.8000,-139.0333,78.26,75,15,17.18,few clouds
15,Bengkulu,ID,-3.8004,102.2655,78.01,77,97,3.74,overcast clouds
20,Saint-Georges,GD,12.0564,-61.7485,82.08,78,20,11.50,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,86.32,67,1,4.00,clear sky
3,Rikitea,PF,-23.1203,-134.9692,74.68,79,54,14.63,light rain
4,Marsa Matruh,EG,31.3525,27.2453,75.40,78,0,3.44,clear sky
5,Hue,VN,16.4667,107.6000,86.11,66,20,5.75,few clouds
7,Vaini,TO,-21.2000,-175.2000,75.36,73,40,11.50,scattered clouds


In [7]:
clean_travel_cities.count()

City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [8]:
clean_travel_cities.columns

Index(['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Current Description'],
      dtype='object')

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,86.32,clear sky,22.0752,-159.3190,
3,Rikitea,PF,74.68,light rain,-23.1203,-134.9692,
4,Marsa Matruh,EG,75.40,clear sky,31.3525,27.2453,
5,Hue,VN,86.11,few clouds,16.4667,107.6000,
7,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,
12,Takoradi,GH,75.31,overcast clouds,4.8845,-1.7554,
13,Labuhan,ID,80.35,broken clouds,-6.8844,112.2051,
14,Atuona,PF,78.26,few clouds,-9.8000,-139.0333,
15,Bengkulu,ID,78.01,overcast clouds,-3.8004,102.2655,
20,Saint-Georges,GD,82.08,few clouds,12.0564,-61.7485,


In [10]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,86.32,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Rikitea,PF,74.68,light rain,-23.1203,-134.9692,People ThankYou
4,Marsa Matruh,EG,75.40,clear sky,31.3525,27.2453,Beau Site
5,Hue,VN,86.11,few clouds,16.4667,107.6000,Saigon Morin Hotel
7,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Takoradi,GH,75.31,overcast clouds,4.8845,-1.7554,Raybow International Hotel
13,Labuhan,ID,80.35,broken clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C
14,Atuona,PF,78.26,few clouds,-9.8000,-139.0333,Villa Enata
15,Bengkulu,ID,78.01,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
20,Saint-Georges,GD,82.08,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,86.32,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Rikitea,PF,74.68,light rain,-23.1203,-134.9692,People ThankYou
4,Marsa Matruh,EG,75.40,clear sky,31.3525,27.2453,Beau Site
5,Hue,VN,86.11,few clouds,16.4667,107.6000,Saigon Morin Hotel
7,Vaini,TO,75.36,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Takoradi,GH,75.31,overcast clouds,4.8845,-1.7554,Raybow International Hotel
13,Labuhan,ID,80.35,broken clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C
14,Atuona,PF,78.26,few clouds,-9.8000,-139.0333,Villa Enata
15,Bengkulu,ID,78.01,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
20,Saint-Georges,GD,82.08,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort


In [13]:
 # 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
 # 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))